# Visualization of application results

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import palettable
import math
from tqdm import tqdm
from sklearn.linear_model import LinearRegression

# Read data

In [ ]:
sim_selection_point = pd.read_csv('./midsave/simulation_selection_popcount.csv').rename(
    columns = {'Rsquared' : 'r2', 'adj.Rsquared' : 'ar2', 'OOC' : 'ooc'})
sim_prediction_point_raw = pd.read_csv('./midsave/simulation_prediction_popcount.csv').rename(
    columns = {'com' : 'MAP_ID'})
map_commune = gpd.read_file('./midsave/map_commune.shp')
map_urbanity = gpd.read_file('./midsave/map_urbanity.shp')

In [ ]:
sim_prediction_point_raw.shape

In [ ]:
sim_prediction_point_raw['MAP_ID'] = sim_prediction_point_raw['MAP_ID'] - 1

### Performance measures

In [ ]:
sim_prediction_point_raw['bias'] = sim_prediction_point_raw['pred'] - sim_prediction_point_raw['true']
sim_prediction_point_raw['bias_benchmark'] = sim_prediction_point_raw['smpl'] - sim_prediction_point_raw['true']
sim_prediction_point_raw['rmse'] = (sim_prediction_point_raw['pred'] - sim_prediction_point_raw['true'])**2
sim_prediction_point_raw['rmse_benchmark'] = (sim_prediction_point_raw['smpl'] - sim_prediction_point_raw['true'])**2

### Select urban/rural

In [ ]:
sim_prediction_point_raw = sim_prediction_point_raw.merge(map_commune[['MAP_ID', 'STATUT']], on = 'MAP_ID', how = 'left')

### Correlation

In [ ]:
sim_prediction_point_correlation = pd.DataFrame({'Model' : (sim_prediction_point_raw.groupby('model')['pred', 'true'].corr()).reset_index().iloc[[1,3,5,7,9,11,13,15], 0], #47
              'Corr' : np.round((sim_prediction_point_raw.groupby('model')['pred', 'true'].corr()).reset_index().iloc[[1,3,5,7,9,11,13,15], 2], decimals = 3),  #47
              'Corr_In' : np.round((sim_prediction_point_raw.loc[(sim_prediction_point_raw['type'] == 'in')].groupby('model')['pred', 'true'].corr()).reset_index().iloc[[1,3,5,7,9,11,13,15], 2], decimals = 3), #47
              'Corr_Out' : np.round((sim_prediction_point_raw.loc[(sim_prediction_point_raw['type'] == 'out')].groupby('model')['pred', 'true'].corr()).reset_index().iloc[[1,3,5,7,9,11,13,15], 2], decimals = 3), #47
              'Corr_Kriged' : np.round((sim_prediction_point_raw.loc[(sim_prediction_point_raw['type'] == 'kriged')].groupby('model')['pred', 'true'].corr()).reset_index().iloc[[1,3,5,7], 2], decimals = 3) #23
             })

### Sample size

In [ ]:
sim_prediction_point_n = pd.DataFrame({'N' : ((sim_prediction_point_raw.groupby('model')['pred'].count().rename('n'))/500).astype('uint16'),
              'N_In' : np.round((sim_prediction_point_raw[sim_prediction_point_raw['type'] == 'in'].groupby('model')['pred'].count())/500, decimals = 0).astype('uint16'),
              'N_Out' : np.round((sim_prediction_point_raw[sim_prediction_point_raw['type'] == 'out'].groupby('model')['pred'].count())/500, decimals = 0).astype('uint16'),
              'N_Kriged' : np.round((sim_prediction_point_raw[sim_prediction_point_raw['type'] == 'kriged'].groupby('model')['pred'].count())/500, decimals = 0).astype('uint16')
             })

In [ ]:
sim_prediction_point_correlation.sort_values(by=['Corr_Out'])

In [ ]:
sim_prediction_point_correlation.to_latex()

In [ ]:
sim_prediction_point_n

In [ ]:
sim_prediction_point_n.to_latex()

### Correlation by urban status

In [ ]:
plt.subplots(1, figsize=(10, 10))

ax = sns.scatterplot(x="true", y="pred", 
              hue="STATUT",
              data=sim_prediction_point_raw.loc[(sim_prediction_point_raw['run'] == 200) & (sim_prediction_point_raw['model'] == 'point_nocount') ],
                palette = 'rocket',
                s = 100,
                legend = 'full')

sns.despine()

ax.set_xlim([-0.01, 1.01])
ax.set_ylim([-0.01, 1.01])
ax.set_xticks([0.2, 0.4, 0.6, 0.8, 1])
ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1])

plt.xlabel('True unemployment rate', fontsize = 14)
plt.ylabel('Estimated unemployment rate', fontsize = 14)
ax.tick_params(axis='both', which='major', labelsize=14)

plt.plot([0, 0.8], [0, 0.8], linewidth=1, c = "black")

legend_labels, _= ax.get_legend_handles_labels()
plt.legend(legend_labels, ['','Rural', 'Urban'], title='Commune Status', loc='upper right', frameon=False, fontsize = 14, title_fontsize = 14)

plt.savefig('./Replication/data/Outputs/application_worknoold_scatter_by_urbanity.png', transparent=False, pad_inches=0)

plt.show()


In [ ]:
sim_prediction_point_correlation_urban = pd.DataFrame({'Model' : (sim_prediction_point_raw.groupby('model')['pred', 'true'].corr()).reset_index().iloc[[1,3,5,7,9,11,13,15], 0],
              'Corr' : np.round((sim_prediction_point_raw.groupby('model')['pred', 'true'].corr()).reset_index().iloc[[1,3,5,7,9,11,13,15], 2], decimals = 3),
              'Corr_Rural' : np.round((sim_prediction_point_raw.loc[(sim_prediction_point_raw['STATUT'] == 'CR')].groupby('model')['pred', 'true'].corr()).reset_index().iloc[[1,3,5,7,9,11,13,15], 2], decimals = 3),
              'Corr_Urban' : np.round((sim_prediction_point_raw.loc[(sim_prediction_point_raw['STATUT'] == 'CA')].groupby('model')['pred', 'true'].corr()).reset_index().iloc[[1,3,5,7,9,11,13,15], 2], decimals = 3)
             })

In [ ]:
sim_prediction_point_correlation_urban.to_latex()

In [ ]:
sim_prediction_point_n_urban = pd.DataFrame({'N' : ((sim_prediction_point_raw.groupby('model')['pred'].count().rename('n'))/500).astype('uint16'),
              'N_Rural' : np.round((sim_prediction_point_raw[sim_prediction_point_raw['STATUT'] == 'CR'].groupby('model')['pred'].count())/500, decimals = 0).astype('uint16'),
              'N_Urban' : np.round((sim_prediction_point_raw[sim_prediction_point_raw['STATUT'] == 'CA'].groupby('model')['pred'].count())/500, decimals = 0).astype('uint16')
             })

In [ ]:
sim_prediction_point_n_urban

# Prediction performance

### Ranking

adj.R²

In [ ]:
sim_selection_point_rank = sim_selection_point[(sim_selection_point['model'] == 'point_nocount')
                    |(sim_selection_point['model'] == 'voronoi_nocount')
                    |(sim_selection_point['model'] == 'augvoronoi_nocount')
                    |(sim_selection_point['model'] == 'overlap_best_nocount')
                    |(sim_selection_point['model'] == 'overlap_knn_nocount')
                    |(sim_selection_point['model'] == 'wpgvoronoi_nocount')
                    |(sim_selection_point['model'] == 'wpg_overlap_best_nocount')
                    |(sim_selection_point['model'] == 'wpg_overlap_knn_nocount')].copy()

In [ ]:
sim_selection_point_rank['best_ar2'] = 0
sim_selection_point_rank.loc[sim_selection_point_rank.groupby('run')['ar2'].transform(max) == sim_selection_point_rank.ar2, 'best_ar2'] = 1

In [ ]:
sim_selection_point_rank.groupby('model')['best_ar2'].mean().reset_index().sort_values(by=['best_ar2'])

In [ ]:
sim_selection_point_rank.groupby('model')['best_ar2'].mean().reset_index().sort_values(by=['best_ar2']).to_latex()

Bias

In [ ]:
sim_prediction_point_rank = sim_prediction_point_raw[(sim_prediction_point_raw['model'] == 'point_nocount')
                    |(sim_prediction_point_raw['model'] == 'voronoi_nocount')
                    |(sim_prediction_point_raw['model'] == 'augvoronoi_nocount')
                    |(sim_prediction_point_raw['model'] == 'overlap_best_nocount')
                    |(sim_prediction_point_raw['model'] == 'overlap_knn_nocount')
                    |(sim_prediction_point_raw['model'] == 'wpgvoronoi_nocount')
                    |(sim_prediction_point_raw['model'] == 'wpg_overlap_best_nocount')
                    |(sim_prediction_point_raw['model'] == 'wpg_overlap_knn_nocount')]

In [ ]:
sim_prediction_point_rank = sim_prediction_point_rank.groupby(['model', 'run', 'type'])[['bias', 'rmse']].mean().reset_index()

In [ ]:
sim_prediction_point_rank['bias'] = abs(sim_prediction_point_rank['bias'])
sim_prediction_point_rank['rmse'] = np.sqrt(sim_prediction_point_rank['rmse'])

In [ ]:
sim_prediction_point_rank['best_bias'] = 0
sim_prediction_point_rank.loc[sim_prediction_point_rank.groupby(['type', 'run'])['bias'].transform(min) == sim_prediction_point_rank.bias, 'best_bias'] = 1

In [ ]:
sim_prediction_point_rank.groupby(['type','model'])['best_bias'].mean().reset_index().sort_values(by=['type','best_bias']).to_latex()

RMSE

In [ ]:
sim_prediction_point_rank['best_rmse'] = 0
sim_prediction_point_rank.loc[sim_prediction_point_rank.groupby(['type', 'run'])['rmse'].transform(min) == sim_prediction_point_rank.rmse, 'best_rmse'] = 1

In [ ]:
sim_prediction_point_rank.groupby(['type','model'])['best_rmse'].mean().reset_index().sort_values(by=['type','best_rmse']).to_latex()

Number of predictors

In [ ]:
sim_selection_point_rank.groupby('model')['n_pred'].mean().reset_index().sort_values(by=['n_pred']).to_latex()

### In-Sample

In [ ]:
sim_prediction_point = sim_prediction_point_raw[sim_prediction_point_raw['type'] == 'in'].copy()

In [ ]:
sim_selection_point.loc[sim_selection_point['model'] == 'point_nocount', 'ar2'].shape

R²

In [ ]:
f, ax = plt.subplots(1, figsize=(15, 5))
sns.boxplot(data = pd.DataFrame({
                                 'Point-to-Polygon' : sim_selection_point.loc[sim_selection_point['model'] == 'point_nocount', 'ar2'],
                                 'Voronoi' : sim_selection_point.loc[sim_selection_point['model'] == 'voronoi_nocount', 'ar2'],
                                 'Augmented \n Voronoi (GUF)' : sim_selection_point.loc[sim_selection_point['model'] == 'augvoronoi_nocount', 'ar2'],
                                 'Simple HATA \n (BSA, GUF)' : sim_selection_point.loc[sim_selection_point['model'] == 'overlap_best_nocount', 'ar2'],
                                 'Simple HATA \n (IDW, GUF)' : sim_selection_point.loc[sim_selection_point['model'] == 'overlap_knn_nocount', 'ar2'],
                                 'Augmented \n Voronoi (WPG)' : sim_selection_point.loc[sim_selection_point['model'] == 'wpgvoronoi_nocount', 'ar2'],
                                 'Simple HATA \n (BSA, WPG)' : sim_selection_point.loc[sim_selection_point['model'] == 'wpg_overlap_best_nocount', 'ar2'],
                                 'Simple HATA \n (IDW, WPG)' : sim_selection_point.loc[sim_selection_point['model'] == 'wpg_overlap_knn_nocount', 'ar2'],
                                }),
            palette = 'rocket')

#ax.set(ylim=(0, 1))
ax.tick_params(labelsize=14)

plt.savefig('./Replication/data/Outputs/application_boxplot_ar2_in_worknoold_wpg.png', transparent=False, pad_inches=0)

plt.show()

Bias

In [ ]:
f, ax = plt.subplots(1, figsize=(15, 5))
sns.boxplot(data = pd.DataFrame({
                                 'Direct' : sim_prediction_point.groupby('run')['bias_benchmark'].mean(),
                                 'Point-to-Polygon' : sim_prediction_point[sim_prediction_point['model'] == 'point_nocount'].groupby('run')['bias'].mean(),
                                 'Voronoi' : sim_prediction_point[sim_prediction_point['model'] == 'voronoi_nocount'].groupby('run')['bias'].mean(),
                                 'Augmented \n Voronoi (GUF)' : sim_prediction_point[sim_prediction_point['model'] == 'augvoronoi_nocount'].groupby('run')['bias'].mean(),
                                 'Simple HATA \n (BSA, GUF)' : sim_prediction_point[sim_prediction_point['model'] == 'overlap_best_nocount'].groupby('run')['bias'].mean(),
                                 'Simple HATA \n (IDW, GUF)' : sim_prediction_point[sim_prediction_point['model'] == 'overlap_knn_nocount'].groupby('run')['bias'].mean(),
                                 'Augmented \n Voronoi (WPG)' : sim_prediction_point[sim_prediction_point['model'] == 'wpgvoronoi_nocount'].groupby('MAP_ID')['bias'].mean(),
                                 'Simple HATA \n (BSA, WPG)' : sim_prediction_point[sim_prediction_point['model'] == 'wpg_overlap_best_nocount'].groupby('MAP_ID')['bias'].mean(),
                                 'Simple HATA \n (IDW, WPG)' : sim_prediction_point[sim_prediction_point['model'] == 'wpg_overlap_knn_nocount'].groupby('MAP_ID')['bias'].mean(),
                                }),
            palette = 'rocket')

#ax.set(ylim=(0, 1))
ax.tick_params(labelsize=14)

plt.savefig('./Replication/data/Outputs/application_boxplot_bias_in_worknoold_wpg.png', transparent=False, pad_inches=0)

plt.show()

RMSE

In [ ]:
f, ax = plt.subplots(1, figsize=(15, 5))
sns.boxplot(data = pd.DataFrame({'Direct' : np.sqrt(sim_prediction_point.groupby('run')['rmse_benchmark'].mean()),
                                 'Point-to-Polygon' : np.sqrt(sim_prediction_point[sim_prediction_point['model'] == 'point_nocount'].groupby('run')['rmse'].mean()),
                                 'Voronoi' : np.sqrt(sim_prediction_point[sim_prediction_point['model'] == 'voronoi_nocount'].groupby('run')['rmse'].mean()),
                                 'Augmented \n Voronoi (GUF)' : np.sqrt(sim_prediction_point[sim_prediction_point['model'] == 'augvoronoi_nocount'].groupby('run')['rmse'].mean()),
                                 'Simple HATA \n (BSA, GUF)' : np.sqrt(sim_prediction_point[sim_prediction_point['model'] == 'overlap_best_nocount'].groupby('run')['rmse'].mean()),
                                 'Simple HATA \n (IDW, GUF)' : np.sqrt(sim_prediction_point[sim_prediction_point['model'] == 'overlap_knn_nocount'].groupby('run')['rmse'].mean()),
                                 'Augmented \n Voronoi (WPG)' : np.sqrt(sim_prediction_point[sim_prediction_point['model'] == 'wpgvoronoi_nocount'].groupby('MAP_ID')['rmse'].mean()),
                                 'Simple HATA \n (BSA, WPG)' : np.sqrt(sim_prediction_point[sim_prediction_point['model'] == 'wpg_overlap_best_nocount'].groupby('MAP_ID')['rmse'].mean()),
                                 'Simple HATA \n (IDW, WPG)' : np.sqrt(sim_prediction_point[sim_prediction_point['model'] == 'wpg_overlap_knn_nocount'].groupby('MAP_ID')['rmse'].mean()),
                                }),
            palette = 'rocket')

#ax.set(ylim=(0, 1))
ax.tick_params(labelsize=14)

plt.savefig('./Replication/data/Outputs/application_boxplot_rmse_in_worknoold_wpg.png', transparent=False, pad_inches=0)

plt.show()

### OOS

In [ ]:
sim_prediction_point = sim_prediction_point_raw[sim_prediction_point_raw['type'] == 'out']

Bias

In [ ]:
f, ax = plt.subplots(1, figsize=(15, 5))
sns.boxplot(data = pd.DataFrame({'Point-to-Polygon' : sim_prediction_point[sim_prediction_point['model'] == 'point_nocount'].groupby('run')['bias'].mean(),
                                 'Voronoi' : sim_prediction_point[sim_prediction_point['model'] == 'voronoi_nocount'].groupby('run')['bias'].mean(),
                                 'Augmented \n Voronoi (GUF)' : sim_prediction_point[sim_prediction_point['model'] == 'augvoronoi_nocount'].groupby('run')['bias'].mean(),
                                 'Simple HATA \n (BSA, GUF)' : sim_prediction_point[sim_prediction_point['model'] == 'overlap_best_nocount'].groupby('run')['bias'].mean(),
                                 'Simple HATA \n (IDW, GUF)' : sim_prediction_point[sim_prediction_point['model'] == 'overlap_knn_nocount'].groupby('run')['bias'].mean(),
                                 'Augmented \n Voronoi (WPG)' : sim_prediction_point[sim_prediction_point['model'] == 'wpgvoronoi_nocount'].groupby('MAP_ID')['bias'].mean(),
                                 'Simple HATA \n (BSA, WPG)' : sim_prediction_point[sim_prediction_point['model'] == 'wpg_overlap_best_nocount'].groupby('MAP_ID')['bias'].mean(),
                                 'Simple HATA \n (IDW, WPG)' : sim_prediction_point[sim_prediction_point['model'] == 'wpg_overlap_knn_nocount'].groupby('MAP_ID')['bias'].mean()
                                }),
            palette = 'rocket')

#ax.set(ylim=(0, 1))
ax.tick_params(labelsize=14)

plt.savefig('./Replication/data/Outputs/application_boxplot_bias_out_worknoold.png', transparent=False, pad_inches=0)

plt.show()

RMSE

In [ ]:
f, ax = plt.subplots(1, figsize=(15, 5))
sns.boxplot(data = pd.DataFrame({'Point-to-Polygon' : np.sqrt(sim_prediction_point[sim_prediction_point['model'] == 'point_nocount'].groupby('run')['rmse'].mean()),
                                 'Voronoi' : np.sqrt(sim_prediction_point[sim_prediction_point['model'] == 'voronoi_nocount'].groupby('run')['rmse'].mean()),
                                 'Augmented \n Voronoi (GUF)' : np.sqrt(sim_prediction_point[sim_prediction_point['model'] == 'augvoronoi_nocount'].groupby('run')['rmse'].mean()),
                                 'Simple HATA \n (BSA, GUF)' : np.sqrt(sim_prediction_point[sim_prediction_point['model'] == 'overlap_best_nocount'].groupby('run')['rmse'].mean()),
                                 'Simple HATA \n (IDW, GUF)' : np.sqrt(sim_prediction_point[sim_prediction_point['model'] == 'overlap_knn_nocount'].groupby('run')['rmse'].mean()),
                                 'Augmented \n Voronoi (WPG)' : np.sqrt(sim_prediction_point[sim_prediction_point['model'] == 'wpgvoronoi_nocount'].groupby('MAP_ID')['rmse'].mean()),
                                 'Simple HATA \n (BSA, WPG)' : np.sqrt(sim_prediction_point[sim_prediction_point['model'] == 'wpg_overlap_best_nocount'].groupby('MAP_ID')['rmse'].mean()),
                                 'Simple HATA \n (IDW, WPG)' : np.sqrt(sim_prediction_point[sim_prediction_point['model'] == 'wpg_overlap_knn_nocount'].groupby('MAP_ID')['rmse'].mean())
                                }),
            palette = 'rocket')

#ax.set(ylim=(0, 1))
ax.tick_params(labelsize=14)

plt.savefig('./Replication/data/Outputs/application_boxplot_rmse_out_worknoold.png', transparent=False, pad_inches=0)

plt.show()

### Map

In [ ]:
map_commune = map_commune.merge(sim_prediction_point_raw.groupby('MAP_ID')['true'].mean().reset_index(), on = 'MAP_ID', how = 'left')

In [ ]:
for i in sim_prediction_point_raw['model'].unique():
    
    df = sim_prediction_point_raw.loc[sim_prediction_point_raw['model'] == i].groupby('MAP_ID')['bias'].mean().rename(str('bias_'+i)).reset_index()
    
    map_commune = map_commune.merge(df, on = 'MAP_ID', how = 'left')

In [ ]:
map_commune.describe()

In [ ]:
vmin = -0.4
vmax = 0.4

In [ ]:
f, ((ax2, ax3), (ax4, ax5)) = plt.subplots(2, 2, figsize=(10, 5))

# map_commune[['geometry', 'true']].dropna().plot(ax = ax1, column = 'true', cmap = plt.get_cmap("rocket"), vmin = vmin, vmax = vmax, linewidth = 0.8, edgecolor = 'black')
# ax1.set_title('True Unemployment Rate')
# map_commune[['geometry', 'smpl']].dropna().plot(ax = ax1, column = 'smpl', cmap = plt.get_cmap("rocket"), vmin = vmin, vmax = vmax, linewidth = 0.8, edgecolor = 'black')
# ax1.set_title('Direct')
map_commune[['geometry', 'bias_point_nocount']].dropna().plot(ax = ax2, column = 'bias_point_nocount', cmap = plt.get_cmap("rocket"), vmin = vmin, vmax = vmax, linewidth = 0.8, edgecolor = 'black')
ax2.set_title('Point-to-Polygon')
map_commune[['geometry', 'bias_voronoi_nocount']].dropna().plot(ax = ax3, column = 'bias_voronoi_nocount', cmap = plt.get_cmap("rocket"), vmin = vmin, vmax = vmax, linewidth = 0.8, edgecolor = 'black')
ax3.set_title('Voronoi')
map_commune[['geometry', 'bias_augvoronoi_nocount']].dropna().plot(ax = ax4, column = 'bias_augvoronoi_nocount', cmap = plt.get_cmap("rocket"), vmin = vmin, vmax = vmax, linewidth = 0.8, edgecolor = 'black')
ax4.set_title('Augmented Voronoi')
map_commune[['geometry', 'bias_overlap_best_nocount']].dropna().plot(ax = ax5, column = 'bias_overlap_best_nocount', cmap = plt.get_cmap("rocket"), vmin = vmin, vmax = vmax, linewidth = 0.8, edgecolor = 'black')
ax5.set_title('HATA (BSA)')

for ax in f.get_axes():
    ax.label_outer()

    
f.subplots_adjust(right=0.8)
im = plt.cm.ScalarMappable(cmap = plt.get_cmap("rocket"), norm = plt.Normalize(vmin = vmin, vmax = vmax))
im._A = []
#cbar = fig.colorbar(sm)
cbar_ax = f.add_axes([0.85, 0.15, 0.05, 0.7])
f.colorbar(im, cax=cbar_ax)

plt.savefig('./Replication/data/Outputs/unemployment_rate_by_com.png', transparent=False, pad_inches=0)

plt.show()

# Coverage maps

### Settlements

In [ ]:
map_pixel = gpd.read_file('./data/map_pixel_wpg.shp')

In [ ]:
map_pixel_adm = gpd.tools.sjoin(map_pixel, map_commune[['MAP_ID', 'geometry']], how="left", op='intersects').drop(columns=['index_right'])

In [ ]:
map_pixel_adm.drop_duplicates(subset = 'pixel_id', keep = False, inplace = True)

In [ ]:
map_pixel_adm = map_pixel_adm.dropna()

In [ ]:
map_pixel_adm.head(3)

In [ ]:
f, ax = plt.subplots(1, figsize=(10, 10))
map_commune.plot(ax=ax, color = '#701f57', edgecolor='#701f57', linewidth = 2)
plt.scatter(map_pixel_adm.geometry.x, map_pixel_adm.geometry.y, marker='.', s=0.1, color='white', alpha=1)
plt.axis('off')

plt.savefig('./Replication/data/Outputs/application_settlements_wpg.png', transparent=True, pad_inches=0)

plt.show()

### Direct survey estimate

In [ ]:
map_direct = map_commune[map_commune.MAP_ID.isin(sim_prediction_point_raw.loc[sim_prediction_point_raw['type'] == 'in', 'MAP_ID'].unique())]

In [ ]:
f, ax = plt.subplots(1, figsize=(10, 10))
map_direct.plot(ax=ax, facecolor = '#701f57', edgecolor='white', alpha=1)
plt.axis('off')

plt.savefig('./Replication/data/Outputs/application_direct_worknoold.png', transparent=True, pad_inches=0)

plt.show()

### Point-to-Polygon

In [ ]:
ant_details = gpd.read_file('./data/ant_details.shp')

In [ ]:
len(ant_details.MAP_ID.unique())

In [ ]:
map_p2p = map_commune[map_commune.MAP_ID.isin(ant_details.MAP_ID.unique())]

In [ ]:
f, ax = plt.subplots(1, figsize=(10, 10))
map_p2p.plot(ax=ax, facecolor = '#701f57', edgecolor='white', alpha=1)
plt.axis('off')

plt.savefig('./Replication/data/Outputs/application_p2p_worknoold.png', transparent=True, pad_inches=0)

plt.show()

### Voronoi

In [ ]:
map_voronoi = gpd.read_file('./data/map_voronoi_points.shp')

In [ ]:
f, ax = plt.subplots(1, figsize=(10, 10))
map_voronoi.plot(ax=ax, facecolor = '#701f57', edgecolor='white', alpha=1)
plt.axis('off')

plt.savefig('./Replication/data/Outputs/application_voronoi.png', transparent=True, pad_inches=0)

plt.show()

### HATA

In [ ]:
ant_details = gpd.read_file('./data/ant_details.shp')

In [ ]:
%%capture
%run ./propagation_model_extended_hata.ipynb

In [ ]:
ant_details.head(3)

In [ ]:
for i in tqdm(ant_details.index):
    for j in np.arange(0, 100, 0.01):
        alpha = alpha_value(ant_details.loc[i, 'frequency'], j, ant_details.loc[i, 'antenna_he'])
        path_loss = calculate_path_loss_hata(ant_details.loc[i, 'frequency'], ant_details.loc[i, 'settlement'], j, ant_details.loc[i, 'antenna_he'], 1, alpha)

        if (45 - path_loss) < -110:
            ant_details.loc[i, 'distance'] = j * 10**3
            break

In [ ]:
ant_details = ant_details[ant_details['distance'] != 0]

In [ ]:
ant_details_utm = ant_details.to_crs({'init': 'epsg:31028'})

In [ ]:
ant_details_utm['geometry'] = ant_details_utm.apply(lambda x : x.geometry.buffer(x.distance), axis = 1)

In [ ]:
ant_details_utm['bts_km2'] = ant_details_utm.area / 10**6

In [ ]:
map_ant_details = ant_details_utm.to_crs({'init': 'epsg:4326'})

HATA

In [ ]:
f, ax = plt.subplots(1, figsize=(10, 10))
map_ant_details.plot(ax=ax, color = '#701f57', alpha=0.4, edgecolor='white')
plt.axis('off')

plt.savefig('./Replication/data/Outputs/application_hata.png', transparent=True, pad_inches=0)

plt.show()

2G

In [ ]:
f, ax = plt.subplots(1, figsize=(10, 10))
map_ant_details[map_ant_details.frequency == 900].plot(ax=ax, color = '#701f57', alpha=0.4)
map_commune.plot(ax=ax, facecolor = 'None', edgecolor='white', alpha=0.7)
plt.axis('off')

plt.savefig('./Replication/data/Outputs/application_hata_2g.png', transparent=True, pad_inches=0)

plt.show()

3G

In [ ]:
f, ax = plt.subplots(1, figsize=(10, 10))
map_ant_details[map_ant_details.frequency == 2100].plot(ax=ax, color = '#701f57', alpha=0.4)
map_commune.plot(ax=ax, facecolor = 'None', edgecolor='white', alpha=0.7)
plt.axis('off')

plt.savefig('./Replication/data/Outputs/application_hata_3g.png', transparent=True, pad_inches=0)

plt.show()

### General performance measures

In [ ]:
# results_final = pd.DataFrame()

# for m in tqdm(sim_prediction_point_raw['model'].unique()):
    
#     results = pd.DataFrame()
    
#     temp = sim_prediction_point_raw.loc[(sim_prediction_point_raw['model'] == m), ['true', 'pred', 'type','run']].dropna().copy()
    
#     for j in temp['type'].unique():
        
#         result = pd.DataFrame()
        
#         temp2 = temp.loc[(temp['type'] == j)].dropna().copy()
        
#         for i in temp2['run'].unique():
            
#             temp3 = temp2.loc[(temp['run'] == i)].dropna().copy()

#             result.loc[i, 'model'] = m
#             result.loc[i, 'type'] = j
#             result.loc[i, 'run'] = i
#             result.loc[i, 'bias'] = (temp3['pred'] - temp3['true']).mean()
#             result.loc[i, 'rmse'] = np.sqrt(((temp3['pred'] - temp3['true'])**2).mean())

#             temp4 = temp3[['true', 'pred']]
#             x = np.array(temp4['pred']).reshape((-1, 1))
#             y = np.array(temp4['true'])

#             result.loc[i, 'r2'] = LinearRegression().fit(x, y).score(x, y)
            
#         results = results.append(result)
        
#     results_final = results_final.append(results)

### Population variation

In [ ]:
map_pixel_adm = pd.read_csv('./data/dem_proj_grid_old.csv')

In [ ]:
pop_var = map_pixel_adm.groupby('MAP_ID')['pop_per_grid'].var()

In [ ]:
pop_var.describe()

In [ ]:
len(pop_var[pop_var > 100])

In [ ]:
pop_var_censored = pop_var[pop_var < 10]

In [ ]:
f, ax = plt.subplots(1, figsize=(5, 20))
sns.boxplot(data = pd.DataFrame({'Point-to-Polygon' : pop_var_censored
                                }),
            palette = 'rocket')

ax.tick_params(labelsize=14)

plt.savefig('./Replication/data/Outputs/application_boxplot_ar2_in.png', transparent=False, pad_inches=0)

plt.show()